In [10]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [11]:
Today = '31/08/24'
Today_1 = '2024-08-31'
Today_2 = '31/08/24'
# Today_1 and TODAY_2!!!

print_Debt = True
print_Deriv = True
excel_tofolder_on_Z = True

In [17]:
query = f"""
select * from 
(
select "reportDate","effectiveFrom","entity","dealNo","holding","counterparty","amount","eventCurrency","amountUSD","actionDate","dealClass1","cpartyType", "dealClass2","instrumentOwner","dealSet","facility", 
MAX("effectiveFrom") OVER (partition BY "reportDate") as "max_effectiveFrom"
from "RISKACCESS"."quantumDebt" 
where "reportDate" = TO_DATE('{Today}', 'DD/MM/YY')
)
where "effectiveFrom" = "max_effectiveFrom"
"""
data_Debt_export = export_from_RISKCUSTOM(query)
data_Debt_export.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(7340, 17)

Фильтрация данных

In [18]:
for i in ['SUEK', 'EuroChem']:
    data_Debt_work = data_Debt_export[data_Debt_export.holding==i].reset_index(drop=True) # Фильтрация данных по столбцу holding (переменная Group) и effectiveFrom
    data_Debt_work = data_Debt_work[~data_Debt_work.entity.isna()]
    data_Debt_work.entity = data_Debt_work.entity.replace({'RUBND': 'RUSBO'}).reset_index(drop=True)
    if i == 'SUEK':
        data_Debt_work = data_Debt_work.rename(columns={'entity': 'pre_entity'})
        data_Debt_work['entity'] = merge_Mapping(data_Debt_work, col='pre_entity')

    data_Debt_work['Entity_group'] = merge_SalesUnits(data_Debt_work, col='entity', merge_col='ocpSegment').fillna('External')

    # Выполнение цикла для рассчета amount, amountUSD и присваивания значения counterparty для каждого DealSet

    deals_data = data_Debt_work[['entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']] # Выбор нужного разреза данных
    deals_data_Bonds = deals_data[deals_data.dealSet.str.contains('Bond')] # Фильтрация разреза по значениям в DealSet, которые содержат слово Bond

    Bonds_list = deals_data_Bonds.dealSet.unique()

    for Bond in Bonds_list:
        try:
            Bond_data = deals_data_Bonds.query('dealSet ==@Bond') # Отбор данных одного Bond
            assert len(Bond_data[Bond_data.instrumentOwner == 'EC_ISSUED_BONDS']) != 0

            for column in Bond_data[['amount', 'amountUSD']]:
                Traded_data_sum = Bond_data.query('instrumentOwner =="EC_TRADED_BONDS"')[column].sum()
                Issued_data = Bond_data.query('instrumentOwner =="EC_ISSUED_BONDS"')[column]
                # Внесение нового значения amount в issued bonds
                Bond_data.loc[Bond_data['instrumentOwner'] =="EC_ISSUED_BONDS", column] = Traded_data_sum + Issued_data 
            # Внесение нового значения counterparty в traded bonds
            Issied_data_entity = Bond_data.query('instrumentOwner =="EC_ISSUED_BONDS"').Entity.squeeze()
            Bond_data.loc[Bond_data['instrumentOwner'] =="EC_TRADED_BONDS", 'counterparty'] = Issied_data_entity
            deals_data_Bonds.loc[deals_data_Bonds['dealSet'] == Bond] = Bond_data
        except:
            continue

    deals_data.loc[deals_data.dealSet.str.contains('Bond')] = deals_data_Bonds
    data_Debt_work[['entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']] = deals_data
    # Запись выполненного цикла в в основные данные

    data_Debt_work = Period(data_Debt_work, day_for_count=Today, col_with_date='actionDate').reset_index(drop=True)

    data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'] = data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5] 
    # Обрезка " (EC)" в каждой строке

    data_Debt_work = data_Debt_work[~data_Debt_work.counterparty.isna()].reset_index(drop=True)
    data_Debt_work['CompCode'] = merge_Mapping(data_Debt_work, col='counterparty').fillna('External')
    data_Debt_work['Counterparty_Group'] = merge_SalesUnits(data_Debt_work, col='CompCode', merge_col='ocpSegment').fillna('External')
    # Merge counterparty с Mapping и SalesUnits

    data_Debt_work['Source'] = 'Quantum'

    if i == 'SUEK':
        data_Debt_work_SUEK = data_Debt_work
    if i == 'EuroChem':
        data_Debt_work_ECH = data_Debt_work

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SGC' 'SGC' 'SGC' ... 'SGC' 'SGC' 'SGC']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14832\2192577195.py:41: UserWarning: This pattern is interpreted as a regular expression, and has match groups. T

Запись Debt в Excel:

In [14]:
Sheet_in_output_Debt = 'Debt'
Output_Debt_SUEK = "_".join([str(date.today()), 'SUEK_quantum_Debt.xlsx'])
Output_Debt_ECH = "_".join([str(date.today()), 'Ech_quantum_Debt.xlsx'])
if excel_tofolder_on_Z == True:
    Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
    Output_Debt_SUEK = Output_path + Output_Debt_SUEK
    Output_Debt_ECH = Output_path + Output_Debt_ECH

data_Debt_export_for_print_SUEK = data_Debt_work_SUEK[['entity', 'Entity_group','counterparty',\
                                'Counterparty_Group','amount','eventCurrency','amountUSD','actionDate','Days',\
                                'Period','dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]\
                                .rename(columns={'amount': 'amoutn outstanding',\
                                                    'eventCurrency': 'Currency',\
                                                    'amountUSD': 'amount USD eq',\
                                                    'actionDate': 'termEnd'})
data_Debt_export_for_print_ECH = data_Debt_work_ECH[['entity', 'Entity_group','counterparty',\
                                'Counterparty_Group','amount','eventCurrency','amountUSD','actionDate','Days',\
                                'Period','dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]\
                                .rename(columns={'amount': 'amoutn outstanding',\
                                                    'eventCurrency': 'Currency',\
                                                    'amountUSD': 'amount USD eq',\
                                                    'actionDate': 'termEnd'})

if print_Debt == True:
    data_Debt_export_for_print_SUEK.to_excel(Output_Debt_SUEK, sheet_name = Sheet_in_output_Debt, index = False)
    data_Debt_export_for_print_ECH.to_excel(Output_Debt_ECH, sheet_name = Sheet_in_output_Debt, index = False) 

In [27]:
check = data_Debt_work[['entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']]
check = check[check.instrumentOwner.str.contains('BONDS')].reset_index(drop=True)
check[check.dealSet == check.dealSet.unique().tolist()[3]]

entity Entity_group counterparty        amount     amountUSD  \
33  RUEMO       RUFERT       NSD RU -2.500000e+10 -2.741625e+08   

    instrumentOwner                     dealSet  
33  EC_ISSUED_BONDS  MCC RuBond 08 2020-04 R25b

In [19]:
manual_map = data_Debt_work.loc[(data_Debt_work['cpartyType'] == 'Internal') & (data_Debt_work['CompCode'] == 'External'), ['counterparty', 'CompCode', 'Counterparty_Group']].drop_duplicates()
manual_map

manual_map2 = data_Debt_work.loc[data_Debt_work['Entity_group'] == 'External', ['entity', 'Entity_group']].drop_duplicates()
manual_map2

manual_map_print = pd.concat([manual_map, manual_map2], axis=1)
manual_map_print
# manual_map_print.to_excel('Manual_map.xlsx', index=False)

counterparty  CompCode Counterparty_Group entity Entity_group
1149      Corrigo  External           External    NaN          NaN

Работа с Forwards и Swaps:

In [20]:
# query = f"""select * from "RISKACCESS"."quantumForwards" """
# data_Forwards_export = export_from_RISKCUSTOM(query)
# data_Forwards_export[data_Forwards_export.positionDate > '2024-03-01'].positionDate.unique()

query = f"""select * from "RISKACCESS"."quantumForwards" where "positionDate" = TO_DATE('{Today_1}', 'YYYY-MM-DD')"""
data_Forwards_export = export_from_RISKCUSTOM(query)

# query = f"""select * from "RISKACCESS"."quantumSwaps" """
# data_Swaps_export = export_from_RISKCUSTOM(query)
# data_Swaps_export[data_Swaps_export.positionDate > '2024-03-01'].positionDate.unique
query = f"""select * from "RISKACCESS"."quantumSwaps" where "positionDate" = TO_DATE('{Today_2}', 'DD/MM/YY')"""
data_Swaps_export = export_from_RISKCUSTOM(query)

data_Forwards_export.shape, data_Swaps_export.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


((6, 80), (11, 84))

In [21]:
data_work_list = [data_Forwards_export, data_Swaps_export]

for data_Deriv_index in range(len(data_work_list)):
    # Проверка вхождений dealNo в Debt и остановка процесса в случае отсутвия данных:
    data_Deriv = data_work_list[data_Deriv_index]
    dealNo_intersect_list = list(set(data_Deriv['dealNo']).intersection(set(data_Debt_export['dealNo'])))
    data_work_Deriv = data_Deriv[~(data_Deriv.dealNo.isin(dealNo_intersect_list))]
    if len(data_work_Deriv) == 0:
        if data_Deriv_index == 0:
            print('data_Forwards_export is empty')
            data_print_Forwards = pd.DataFrame()
            continue
        else:
            print('data_Swaps_export is empty')
            data_print_Swaps = pd.DataFrame()
            continue
        
    data_work_Deriv['holdingEntity'] = data_work_Deriv['holdingEntity'].replace({'XXTSU': 'SUEK'})

    # Создание входящей пары внизу
    Data_for_loop = data_work_Deriv[['entity','holdingEntity','counterparty','dealNo', 'payFaceValue', 'payFXCurrency', 'recFaceValue', 'recFXCurrency', 'maturityDate']]
    for i in range(0, len(Data_for_loop)):
        row_list = list(Data_for_loop.loc[i])
        Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
        Data_for_loop.loc[i,'payFaceValue'] = Data_for_loop.loc[i,'payFaceValue'] * -1
    Data_for_loop = Data_for_loop.sort_values('dealNo')
    # цикл создания новых строк
    Data_for_loop = Period(Data_for_loop, day_for_count=Today_1, col_with_date='maturityDate')
    # Создание Days и Period
    Data_for_loop = add_in_currency_column(Data_for_loop, CCY_to='USD', col_with_CCY='payFXCurrency', date_is_column=False, col_with_VAL='payFaceValue', DATE=Today_1)
    # создание payFaceValue_in_USD
    new_columns = ['Company', 'Settlement', 'Notional_Amount_(USD)', 'Unnamed', 'source']
    Data_for_loop = Data_for_loop.reindex(columns=(Data_for_loop.columns.tolist() + new_columns))
    Data_for_loop['source'] = 'Quantum'
    # Создание пустых столбцов и столбца ресурса

    if data_Deriv_index == 0:
        data_print_Forwards = Data_for_loop
    else:
        data_print_Swaps = Data_for_loop

data_print_Deriv = pd.concat([data_print_Forwards, data_print_Swaps], axis=0)
# Объединение таблиц вниз

data_print_Deriv_columns = {x:y for x,y in zip(data_print_Deriv.columns.tolist(), [[] for x in data_print_Deriv.columns.tolist()])}
data_print_Deriv_columns['Entity_group']=[]
data_print_Deriv_SUEK = pd.DataFrame(data_print_Deriv_columns)
data_print_Deriv_Ech = pd.DataFrame(data_print_Deriv_columns)
for i in ['SUEK', 'EuroChem']:
    data_print_Deriv_group = data_print_Deriv[data_print_Deriv.holdingEntity == i].reset_index(drop=True)
    if len(data_print_Deriv_group) == 0:
        continue   
    if i == 'SUEK':
        data_print_Deriv_group = data_print_Deriv_group.rename(columns={'entity': 'pre_entity'})
        data_print_Deriv_group['entity'] = merge_Mapping(Data_for_loop, col='pre_entity')
        
    data_print_Deriv_group['Entity_group'] = merge_SalesUnits(data_print_Deriv_group, col='entity', merge_col='ocpSegment').fillna('External')
    data_print_Deriv_group = data_print_Deriv_group[['entity','Entity_group','holdingEntity','Company',\
        'counterparty','dealNo','payFXCurrency','payFaceValue',\
        'payFaceValue_in_USD','Settlement','Notional_Amount_(USD)',\
        'Unnamed','maturityDate','Days','Period','source']]
    if i == 'SUEK':
        data_print_Deriv_SUEK = data_print_Deriv_group
    if i == 'EuroChem':
        data_print_Deriv_Ech = data_print_Deriv_group

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14832\388960480.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14832\388960480.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14832\388960480.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

data_Swaps_export is empty


C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']


Запись Deriv в файл:

In [22]:
Sheet_in_output_Deriv = 'Deriv'
Output_Deriv_SUEK = "_".join([str(date.today()), 'SUEK_quantum_Deriv.xlsx'])
Output_Deriv_Ech = "_".join([str(date.today()), 'Ech_quantum_Deriv.xlsx'])
if excel_tofolder_on_Z == True:
    Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
    Output_Deriv_SUEK = Output_path + Output_Deriv_SUEK
    Output_Deriv_Ech = Output_path + Output_Deriv_Ech

if print_Deriv == True:
    if len(data_print_Deriv_SUEK) != 0:
        data_print_Deriv_SUEK.to_excel(Output_Deriv_SUEK, sheet_name=Sheet_in_output_Deriv, index=False)
    if len(data_print_Deriv_Ech) != 0:
        data_print_Deriv_Ech.to_excel(Output_Deriv_Ech, sheet_name=Sheet_in_output_Deriv, index=False)

In [12]:
assert 2==1

AssertionError: 